In [ ]:
import duckdb
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
import sys

directory_path = os.path.abspath(os.path.join('../utils/'))
if directory_path not in sys.path:
    sys.path.append(directory_path)
from functions import *
from constant import *

# Experiment parameters, need to be set before running this notebook.
EXPERIMENT_ID = [""]
EXPERIMENT_START_TIME = [""]
EVENT_IDS = [""]

In [ ]:
# --- Check input validity and create DB connection --- #
assert len(EXPERIMENT_ID)==len(EXPERIMENT_START_TIME)

# Connect to database.
con = duckdb.connect(database=DUCKDB_PATH, read_only=True)

In [ ]:
# --- Data manipulations --- #

# Retrieve relevant data.
EXP_DATA = pd.DataFrame()
for idx, id in enumerate(EXPERIMENT_ID):
    EXP_DATA = pd.concat([EXP_DATA, retrieve_experiment_df(con, id, EXPERIMENT_START_TIME[idx])])

# Remove superfluous entries from dataframe.
EXP_DATA = filterByEventIds(EXP_DATA, EVENT_IDS)

# Calculate latency for each element.
EXP_DATA['time_diff_in_mins'] = EXP_DATA.apply(lambda x: time_diff_in_minutes(x['event_start_time'], x['event_end_time']), axis=1)

In [ ]:
# --- Plot the data --- #

sns.set(rc={'figure.figsize':(18,6)})
sns.set(font_scale=1.2)

# Plot latency (in order of ids).
sns.lineplot(x='event_id', y='time_diff_in_mins', hue='exp_name', data=EXP_DATA)
plt.legend(loc='upper left')
plt.ylabel("Latency (mins)")
plt.xlabel("Event ID")